In [372]:
import pandas as pd
import numpy as np
import pylab 
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import os, sys

### Instruction
Only change the subcategory_name. No Space allowed !!!!!

In [373]:
subcategory_name = 'GERIATRICS' 

In [374]:
df_exclude = pd.read_csv("../data/Final subscriber/"+ subcategory_name + "_sub_final.csv")

In [375]:
df_sales = pd.read_csv("../data/subcategory/"+ subcategory_name + ".csv")

/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [376]:
df_sales = df_sales[~df_sales.address_hash.isin(df_exclude.index)]

## Change Subcategory Here

In [377]:
# df_sales = df_sales[df_sales['Animal_Species'] == 'DOG']

In [378]:
df_sales['date']=pd.to_datetime(df_sales['order_date']).dt.strftime("%Y-%m-%d")

In [379]:
df_sales_date = df_sales
df_sales_171819 = df_sales_date
df_sales_171819 = df_sales_171819.loc[(df_sales_171819['date'] >= '2017-01-01') \
& (df_sales_171819['date']!='2018-12-12')]

### Getting individual's number of orders and end purchse date

In [380]:
customers_orders = {}
customers_start = {}
customers_end = {}
for index, row in df_sales_171819.iterrows():   
    if row['address_hash'] in customers_orders.keys():
        if row['date'] != customers_end[row['address_hash']]:
            customers_orders[row['address_hash']] += 1 # update the amount of orders
            customers_end[row['address_hash']] = row['date'] # update the last order date
    else:
        customers_orders[row['address_hash']] = 1 
        customers_start[row['address_hash']] = row['date'] # record the start order date
        customers_end[row['address_hash']] = row['date'] # update the last order date

In [381]:
# Transform dictionaries into dataframes and merge them according to keys
df_customers_orders = pd.DataFrame(customers_orders.items(), columns=['customer', 'num_orders'])
df_customers_start = pd.DataFrame(customers_start.items(), columns=['customer', 'date_start'])
df_customers_end = pd.DataFrame(customers_end.items(), columns=['customer', 'date_end'])

In [382]:
df_tmp = pd.merge(df_customers_orders, df_customers_start, on='customer', how ='left')

In [383]:
df = pd.merge(df_tmp, df_customers_end, on='customer', how ='left')

In [384]:
df_start_end_sort = df.sort_values(by=['date_start'])

In [385]:
df_start_end_sort.reset_index(inplace = True)

In [386]:
def f(x):    
    return pd.to_datetime(x['date_end']) 
df_start_end_sort['date_leave'] = df_start_end_sort.apply(f, axis=1)

## Churn Rates

**<center>Churn Rate = Customers Left / (New Customers + Existing Customers)</center>** <br/>

In [387]:

# Getting number of days need to fill the column
from datetime import date
d0 = date(2017, 1, 1)
d1 = date(2020, 2, 10)
delta = d1 - d0

# create an array of dates starting at '2017-01-01', one per day. and set day as index
rng = pd.date_range('2017-01-01', periods=1135, freq='D')
df_calender = pd.DataFrame({ 'Date': rng}) 

df_calender['Date']=pd.to_datetime(df_calender['Date']).dt.strftime("%Y-%m-%d")

df_calender_2 = df_calender[:]


# new_subscriber fill

df_start_end_sort.reset_index(inplace = True)


df_start_end = df_start_end_sort.drop(columns = ['level_0'])


i = 0
for index, row in df_calender_2.iterrows():
    count = 0
    while i < df_start_end.shape[0] and df_start_end.at[i,'date_start'] == row['Date']:
        count += 1
        i += 1
    df_calender_2.at[index,'new_cus'] = count



df_sort_by_date_end = df_start_end.sort_values(by=['date_leave'])


# In[788]:


df_sort_by_date_end.reset_index(inplace = True)


# In[789]:


df_sort_by_date_end['date_leave']=pd.to_datetime(df_sort_by_date_end['date_leave']).dt.strftime("%Y-%m-%d")


# In[790]:


i = 0
for index, row in df_calender_2.iterrows():
    count = 0
    while i < df_sort_by_date_end.shape[0] and df_sort_by_date_end.at[i,'date_leave'] == row['Date']:
        count += 1
        i += 1
    df_calender_2.at[index,'new_cus_churn'] = count



# #### Now calculate the daily existing customer column
# The first one or two months existing subscribers base may not be very accurate due to ignorance of active subscribers coming from the end of 2016. This error will gradually disappear after two months or so. Let's first regard the number of unique subscribers during the first month of 2017 as existing subscribers.

# In[792]:

df_sales_week_initial = df_sales_171819.loc[(df_sales_171819['date'] < '2017-01-08') ]
df_sales_month_initial = df_sales_171819.loc[(df_sales_171819['date'] < '2017-02-01')]
existing_week_initial = df_sales_week_initial['address_hash'].nunique()
existing_month_initial = df_sales_month_initial['address_hash'].nunique()
df_existing_week = df_calender_2[:]
df_existing_month = df_calender_2[:]
df_existing_week.at[0,'existing_customers'] = existing_week_initial
df_existing_month.at[0,'existing_customers'] = existing_month_initial
for i in range(1,df_existing_week.shape[0]):
    df_existing_week.at[i,'existing_customers'] = df_existing_week.at[i-1,'existing_customers']     + df_existing_week.at[i-1,'new_cus'] - df_existing_week.at[i-1,'new_cus_churn']

for i in range(1,df_existing_month.shape[0]):
    df_existing_month.at[i,'existing_customers'] = df_existing_month.at[i-1,'existing_customers']     + df_existing_month.at[i-1,'new_cus'] - df_existing_month.at[i-1,'new_cus_churn']

df_existing_week['Date'] = pd.to_datetime(df_existing_week['Date'], errors='coerce')
df_existing_week['Week'] = df_existing_week['Date'].dt.strftime('%Y-%U')
df_existing_month['Month'] = pd.to_datetime(df_existing_month['Date']).dt.to_period('M')

df_month_exist = df_existing_month.groupby(['Month']).agg({'new_cus_churn':'sum','existing_customers':'mean','new_cus':'sum'})

df_month_exist['monthly_churn_rate'] = df_month_exist['new_cus_churn']/df_month_exist['existing_customers'] 

df_month_exist = df_month_exist.fillna(0)

df_week_exist = df_existing_week.groupby(['Week']).agg({'new_cus_churn':'sum','existing_customers':'mean','new_cus':'sum'})

df_week_exist['weekly_churn_rate'] = df_week_exist['new_cus_churn']/df_week_exist['existing_customers'] 

df_week_exist = df_week_exist.fillna(0)

dfs = {subcategory_name + "_monthly_churn_rate":df_month_exist, subcategory_name + "_weekly_churn_rate":df_week_exist} 

for k, v in dfs.items():
    path_name = '../output/subcategory_churn_rate/customers/'
    if not os.path.exists(path_name):
        os.makedirs(path_name)
    v.to_csv(path_name + '{}.csv'.format(k))



/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/renmengyuan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [388]:
# import pandas as pd
# import xlsxwriter
# import glob
# import os

# writer = pd.ExcelWriter('multi_sheet.xlsx', engine='xlsxwriter')
# folders = next(os.walk('.'))[1]

# for host in folders:
#     Path = os.path.join(os.getcwd(), host)

#     for f in glob.glob(os.path.join(Path, "*.csv")):
#         print(f)
#         df = pd.read_csv(f)
#         df.to_excel(writer, sheet_name=os.path.basename(f)[:31])

# writer.save()